In [2]:
import transformer_lens
import torch

/home/jnb5885/miniconda3/envs/ling/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device_name = "cpu"
if torch.cuda.is_available():
    device_name = "cuda" # CUDA for NVIDIA GPU

device = torch.device(device_name)
print(f"Device: {device_name}")

Device: cuda


In [4]:
# Load a model (eg GPT-2 Small)
model = transformer_lens.HookedTransformer.from_pretrained("gpt2-small")

# Run the model and get logits and activations
logits, activations = model.run_with_cache("Hello World")

config.json: 100%|██████████| 665/665 [00:00<00:00, 4.58MB/s]
model.safetensors: 100%|██████████| 548M/548M [00:01<00:00, 276MB/s] 
generation_config.json: 100%|██████████| 124/124 [00:00<00:00, 1.15MB/s]
tokenizer_config.json: 100%|██████████| 26.0/26.0 [00:00<00:00, 273kB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 14.5MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 3.64MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 11.5MB/s]


Loaded pretrained model gpt2-small into HookedTransformer


In [5]:
for k in activations:
    if "resid" in k:
        print(k)

blocks.0.hook_resid_pre
blocks.0.hook_resid_mid
blocks.0.hook_resid_post
blocks.1.hook_resid_pre
blocks.1.hook_resid_mid
blocks.1.hook_resid_post
blocks.2.hook_resid_pre
blocks.2.hook_resid_mid
blocks.2.hook_resid_post
blocks.3.hook_resid_pre
blocks.3.hook_resid_mid
blocks.3.hook_resid_post
blocks.4.hook_resid_pre
blocks.4.hook_resid_mid
blocks.4.hook_resid_post
blocks.5.hook_resid_pre
blocks.5.hook_resid_mid
blocks.5.hook_resid_post
blocks.6.hook_resid_pre
blocks.6.hook_resid_mid
blocks.6.hook_resid_post
blocks.7.hook_resid_pre
blocks.7.hook_resid_mid
blocks.7.hook_resid_post
blocks.8.hook_resid_pre
blocks.8.hook_resid_mid
blocks.8.hook_resid_post
blocks.9.hook_resid_pre
blocks.9.hook_resid_mid
blocks.9.hook_resid_post
blocks.10.hook_resid_pre
blocks.10.hook_resid_mid
blocks.10.hook_resid_post
blocks.11.hook_resid_pre
blocks.11.hook_resid_mid
blocks.11.hook_resid_post
